In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

from numpy.random import seed
seed(1337)
from tensorflow import random
random.set_seed(42)

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import classification_report, accuracy_score

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dense, Input, Conv1D, MaxPooling1D, Embedding, GlobalMaxPool1D, GlobalMaxPooling1D, SpatialDropout1D, LSTM, Flatten, BatchNormalization, Dropout
import transformers

In [4]:
data = pd.read_csv('/content/drive/MyDrive/capstone/balanced_lemmatized_cleaned_stocktwits.csv', index_col=0)

In [5]:
data.head()

,created_at,body,sentiment,raw_content
14,2020-12-15T14:38:18Z,"['going', 'right', 'support', 'even', 'superio...",0,$MSFT Going right through 214 support as if it...
49,2020-12-15T14:23:04Z,"['nobody', 'gonna', 'buy', 'expensive', 'as', ...",0,$AAPL nobody gonna buy expensive ass iPhones w...
61,2020-12-15T14:12:10Z,"['robinhood', 'peep', 'gonna', 'severely', 'di...",0,$AAPL Robinhood peeps gonna be severely disapp...
103,2020-12-15T13:33:52Z,"['always', 'dump', 'dump', 'dump']",0,$AAPL always dump dump dump.
106,2020-12-15T13:30:10Z,"['turd', 'going', 'anywhere', 'pathetic']",0,$AAPL why is this turd not going anywhere. Thi...


In [ ]:
data[data['sentiment']  == 1].shape

(431285, 4)

In [ ]:
new_bull = data[data['sentiment']  == 1].sample(15000)
new_bear = data[data['sentiment']  == 0].sample(15000)
new_df = pd.concat([new_bull, new_bear], ignore_index=True)
new_df

,created_at,body,sentiment,raw_content
0,2021-05-22T04:57:15Z,"['remember', 'bear', 'came', 'march', 'last', ...",1,$TSLA Remember all the bears that came out Mar...
1,2021-11-03T16:58:36Z,"['let', 'goo']",1,$TSLA Letsss Gooooo
2,2021-04-01T15:15:35Z,"['o', 'k', 'tesla', 'ev', 'car', 'road', 'test...",1,$TSLA o 25K Tesla EV car will be road tested i...
3,2021-01-27T17:25:12Z,"['comon', 'let', 'goo', 'keep', 'going', 'look...",1,$AAPL comon letssss gooo.... keep going dont l...
4,2020-09-09T18:59:48Z,"['close', 'over', 'set', 'u', 'up', 'nicely', ...",1,$TSLA close over $365 and it sets us up nicely...
...,...,...,...,...
29995,2020-11-04T14:50:33Z,"['tesla', 'going', 'red', 'apple', 'follow']",0,$AAPL tesla going red apple to follow
29996,2020-02-26T14:44:17Z,"['many', 'people', 'lose', 'money', 'today', '...",0,$TSLA many people will lose money today. Why? ...
29997,2022-02-08T17:15:25Z,"['under', 'prepandemic', 'high', 'sure', 'go',...",0,$SPY $FB now under prepandemic high. But sure...
29998,2020-09-25T16:38:53Z,"['losing', 'steam', 'might', 'good', 'play', '...",0,$WKHS losing steam. Might be a good play for ...


In [ ]:
data[data['sentiment']  == 0].shape

(431285, 4)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['body'], data['sentiment'], test_size=0.2, random_state=42)

In [8]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (690056,)
X_test: (172514,)
y_train: (690056,)
y_test: (172514,)


In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-cased")
# Maximum sequence size for BERT is 512
def regular_encode(texts, tokenizer, maxlen):
    enc_di = tokenizer.batch_encode_plus(texts, return_token_type_ids = False, padding = 'max_length', max_length = maxlen, truncation=True)
    return np.array(enc_di['input_ids'])

In [13]:
# cache
Xtrain_encoded = regular_encode(X_train.astype('str'), tokenizer, maxlen = 31)
Xtest_encoded = regular_encode(X_test.astype('str'),  tokenizer, maxlen = 31)

In [ ]:
Xtest_encoded

array([[101, 164, 112, ...,   0,   0,   0],
       [101, 164, 112, ...,   0,   0,   0],
       [101, 164, 112, ..., 112, 117, 102],
       ...,
       [101, 164, 112, ...,   0,   0,   0],
       [101, 164, 112, ...,   0,   0,   0],
       [101, 164, 112, ...,   0,   0,   0]])

In [11]:
max_len = 31
input_word_ids = tf.keras.layers.Input(shape = (max_len,), dtype = tf.int32, name = "input_word_ids")
transformer_layer = transformers.TFAutoModel.from_pretrained('bert-base-uncased')
sequence_output = transformer_layer(input_word_ids)[0]

cls_token = sequence_output[:, 0, :]
x = tf.keras.layers.Dense(250, activation = 'relu')(cls_token)
x = tf.keras.layers.Dropout(0.5)(x)
out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(inputs = input_word_ids, outputs = out)
model.compile(tf.keras.optimizers.Adam(learning_rate = 5e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 31)]             0         
                                                                 
 tf_bert_model_1 (TFBertMode  TFBaseModelOutputWithPoo  109482240
 l)                          lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             31, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [18]:
#creating the training and testing dataset.
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE 
train_dataset = (tf.data.Dataset.from_tensor_slices((Xtrain_encoded, y_train)).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))
test_dataset = (tf.data.Dataset.from_tensor_slices((Xtest_encoded, y_test)).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))

In [19]:
train_history = model.fit(train_dataset, validation_data=test_dataset, epochs = 3)

Epoch 1/3


21565/21565 [==============================] - 5459s 251ms/step - loss: 0.5660 - accuracy: 0.6974 - val_loss: 0.5162 - val_accuracy: 0.7394
Epoch 2/3
21565/21565 [==============================] - 5415s 251ms/step - loss: 0.5085 - accuracy: 0.7451 - val_loss: 0.4928 - val_accuracy: 0.7543
Epoch 3/3
21565/21565 [==============================] - 5397s 250ms/step - loss: 0.4811 - accuracy: 0.7634 - val_loss: 0.4881 - val_accuracy: 0.7615


In [ ]:
model.save('capstone_BERT.h5')